# 1.简单的留出验证(hold-out validation).

In [ ]:
num_validation_samples = 10000
# 通常需要打乱数据
np.random.shuffle(data)
# 定义验证集
validation_data = data[:num_validation_samples]
data = data[num_validation_samples:]
# 定义训练集
training_data = data[:]
# 在训练集上训练模型,并在验证集上评估模型
model = get_model()
model.train(training_data)
validation_score = model.evaluate(validation_data)

# 调节参数,重新训练,评估,然后再调节....

# 一旦调节好超参数,通常就在所有非测试数据上从头开始训练最终模型
model = get_model()
model.train(np.concatenate([training_data, validation_data]))
test_score = model.evaluate(test_data)

# 2.K折交叉验证

In [ ]:
k = 4
num_validation_samples = len(data) // k

np.random.shuffle(data)

validation_scores = list()
for fold in range(k):
    # 选择验证数据分区
    validation_data = data[fold*num_validation_samples:(fold+1)*num_validation_samples]
    # 使用剩余数据作为训练数据.注意,+运算符是列表合并,不是求和
    training_data = data[:fold*num_validation_samples] + data[(fold+1)*num_validation_samples:]
    
    model = get_model()
    model.train(training_data)
    validation_score = model.evaluate(validation_data)
    validation_scores.append(validation_score)
    
# 最终验证分数:K折验证分数的平均值
validation_score = np.average(validation_scores)
# 在所有非测试数据上训练最终模型并评估
model = get_model()
model.train(data)
test_score = model.evaluate(test_data)


# 3.带有打乱数据的重复K折验证

In [ ]:
具体做法:多次使用K折验证,在每次将数据划分为K个分区之前都先将数据打乱.最终分数是每次K折验证分数的平均值.

# 4.向模型添加L2权重正则化

In [1]:
from keras import regularizers
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu', input_shape=(10000, )))
model.add(layers.Dense(16, kernel_regularizer=regularizers.l2(0.001), activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


# 5.向IMDB中添加dropout

In [ ]:
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000, )))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))